![AIRBNB](https://www.stevenridercpa.au/wp-content/uploads/2022/09/airbnb-tax.jpeg)

# Obligatorio de Deep Learning 
## Semestre 2 - 2023
-------

## Problema

Se presenta un dataset que contiene información de alojamientos publicados en AirBnB con sus respectivos precios. El tamaño del dataset de train es de 1.5 Gb aproximadamente, y 0.5 Gb el de test. Este cuenta con 84 variables predictoras que se podrán utilizar como consideren adecuado.

El objetivo es asignar el precio correcto a los alojamientos listados. 

Además del dataset se les provee esta notebook conteniendo el script de carga de datos y un modelo baseline que corresponde a una arquitectura feed forward.

------

## Consigna

### A) <u>Participación en Competencia Kaggle</u>:
El objetivo de este punto es participar en la competencia de Kaggle y obtener como mínimo un Mean Absolute Error inferior a 70 puntos. [->Link a la competencia<-](https://www.kaggle.com/t/69c648e3aa214d1f812bf2314c8d4ffa).

### B) <u>Utilización de Grid Search (o equivalente)</u>:
Para cumplir con la busqueda de modelos óptimos se debe realizar un grid search lo más abarcativo y metódico posible. Recomendamos enfáticamente [Weights and Biases](https://wandb.ai/site)

### C) <u>Se debe a su vez investigar e implementar las siguientes técnicas</u>:
#### 1. [Batch Normalization](https://machinelearningmastery.com/how-to-accelerate-learning-of-deep-neural-networks-with-batch-normalization/)
#### 2. [Gradient Normalization y/o Gradient Clipping](https://machinelearningmastery.com/how-to-avoid-exploding-gradients-in-neural-networks-with-gradient-clipping/)


Además como en todas las tareas se evaluará la prolijidad de la entrega, el preprocesamiento de datos, visualizaciones y exploración de técnicas alternativas. 

-------

## 1. Setup
### 1.1 Imports

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split

### 1.2 Seteo de seeds

In [ ]:
np.random.seed(117)
tf.random.set_seed(117)

## 2. Carga de datos

In [ ]:
file_path = './airbnb_data/public_train_data.csv'
df = pd.read_csv(file_path)

##  3. Análisis exploratorio de datos
### 3.1 Dimensiones

In [ ]:
df.shape

### 3.2 Obtener información sobre las columnas y tipos de datos

In [ ]:
df.info()

### 3.3 Visualizar las primeras filas del dataset

In [ ]:
df.head(3)

### 3.4 Estadísticas descriptivas

In [ ]:
df.describe()

In [ ]:
df.columns

## 4. Modelo Baseline

### 4.1 Seleccionar características relevantes

In [ ]:
features = ['Bathrooms', 'Bedrooms']  # Reemplaza con las características relevantes
target = 'Price' 
df = df[[*features, target]]
df.dropna(inplace=True)

In [ ]:
X = df[features]
y = df[target]

### 4.2 Dividir los datos en conjuntos de entrenamiento y prueba

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

### 4.3 Definir el modelo

In [ ]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense


model = Sequential([
    Dense(32, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(1, activation='relu')  # Capa de salida para la predicción del precio
])

model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])

### 4.4 Entrenar

In [ ]:
history = model.fit(X_train, y_train, epochs=5, batch_size=128, validation_split=0.2)

### 4.5 Evaluar en Test

In [ ]:
loss, mae = model.evaluate(X_test, y_test)
print(f'Test Loss: {loss}, Test MAE: {mae}')

## 5 Generación de salida para competencia en Kaggle

In [ ]:
file_path2 = './airbnb_data/private_data_to_predict.csv'
data_for_kaggle = pd.read_csv(file_path2)

In [ ]:
kaggle_results = model.predict(data_for_kaggle[features])
test_ids = data_for_kaggle['id']
test_ids = np.array(test_ids).reshape(-1,1)
output = np.stack((test_ids, kaggle_results), axis=-1)
output = output.reshape([-1, 2])
df = pd.DataFrame(output)
df.columns = ['id','expected']  
df['expected'] = df['expected'].fillna(0)   
df.to_csv("output_to_submit.csv", index = False, index_label = False)


## 6 Ejemplo de uso de Weights and Biases

In [ ]:
from tensorflow.keras.layers import Dropout

def get_model(neurons, optimizer, dropout):
    layers = []
    input_shape = (X_train.shape[1],)
    for n in neurons:
        layers.append(Dense(n, activation = "relu", input_shape = input_shape))
        layers.append(Dropout(dropout))
        input_shape = (n,)
        
    model = Sequential(layers)
    model.compile(optimizer = optimizer, loss='mean_squared_error', metrics=['mae'])
    return model

In [ ]:
# Import the W&B Python Library and log into W&B
import wandb

wandb.login()

#Creamos un proyecto en WandB a través de su interfaz
project = "obligatorio_dl"
entity = "franzmayr"

In [ ]:
import traceback

def run_train():
    try:       
        with wandb.init(config = None, project = project, entity=entity):     
            # initialize model
            config = wandb.config
            print(config)
            model= get_model(config.neurons, config.optimizer, config.dropout)
            tf.keras.backend.clear_session()
            wandb_callback = wandb.keras.WandbCallback()
            model.fit(X_train, y_train, epochs=5, batch_size=128, validation_split=0.2, callbacks=[wandb_callback], max_queue_size=3, workers=2)

    except Exception as e:
        # exit gracefully, so wandb logs the problem
        print(traceback.print_exc(), file=sys.stderr)
        exit(1)

In [ ]:
import pprint

sweep_config = {
'name': 'sweep_example',
'method': 'grid',
'metric': {
    'name': 'val_loss',
    'goal': 'minimize'   
},
'parameters': {
    'dropout':{'value': 0.1},
    'neurons':{
        'values': [[32,2],[64,32,2]]
        },
    'optimizer': {
        'values': ['adam', 'sgd']
        }
}
}

pprint.pprint(sweep_config)

In [ ]:
sweep_id = wandb.sweep(sweep_config, project = project, entity = entity)

In [ ]:
wandb.agent(sweep_id, function = run_train, count=10, project = project, entity = entity)